<a href="https://colab.research.google.com/github/SrTorGonz/Homework-2/blob/main/Homework2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Homework 2

Authors: Daniel Alejandro Torres González & Juan David Plata Garrido


In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler

# Leer archivo Excel
df = pd.read_excel('Diagnostico.xlsx')

# Eliminar filas nulas
df.dropna(inplace=True)

# Cambiar los datos de texto a numéricos (columna GENERO)
df['GENERO'] = df['GENERO'].map({'MASCULINO': 1, 'FEMENINO': 2})

# Separar la última columna (labels) y las demás columnas (features)
features_df = df.iloc[:, :-1]  # Todas las columnas excepto la última
labels_df = df.iloc[:, -1]     # Última columna

# Crear tensores con TensorFlow
features_tensor = tf.convert_to_tensor(features_df, dtype=tf.float32)
labels_tensor = tf.convert_to_tensor(labels_df.values, dtype=tf.float32)

#convertir label a (442,1)
labels_tensor = tf.reshape(labels_tensor, (-1, 1))

# Información de los tensores
print("Forma del tensor features:", features_tensor.shape)
print("Tipo del tensor features:", features_tensor.dtype)
print("Forma del tensor labels:", labels_tensor.shape)
print("Tipo del tensor labels:", labels_tensor.dtype)



Forma del tensor features: (442, 11)
Tipo del tensor features: <dtype: 'float32'>
Forma del tensor labels: (442, 1)
Tipo del tensor labels: <dtype: 'float32'>


**Distribución de datos 80/20**

In [2]:
# Determinar el punto de corte del 80% de los datos
split_index = int(0.8 * len(features_tensor))

# Dividir features y labels
train_features = features_tensor[:split_index]  # 80% inicial
test_features = features_tensor[split_index:]   # 20% restante

train_labels = labels_tensor[:split_index]  # 80% inicial
test_labels = labels_tensor[split_index:]   # 20% restante

# Mostrar información de las particiones
print("Forma de train_features:", train_features.shape)
print("Forma de test_features:", test_features.shape)
print("Forma de train_labels:", train_labels.shape)
print("Forma de test_labels:", test_labels.shape)


Forma de train_features: (353, 11)
Forma de test_features: (89, 11)
Forma de train_labels: (353, 1)
Forma de test_labels: (89, 1)


**Leer dataset por lotes**

In [3]:
# Iteraciones para lectura del dataset en formato TensorFlow
def load_array(data_arrays, batch_size, is_train=True):
    """Construct a TensorFlow data iterator."""
    dataset = tf.data.Dataset.from_tensor_slices(data_arrays)
    if is_train:
        dataset = dataset.shuffle(buffer_size=1000)
    dataset = dataset.batch(batch_size)
    return dataset

batch_size = 64
data_iter = load_array((train_features, train_labels), batch_size)

# Prueba de la lectura por minilotes
next(iter(data_iter))

(<tf.Tensor: shape=(64, 11), dtype=float32, numpy=
 array([[ 48.    ,   2.    , 174.    ,  99.9   , 123.    ,  97.    ,
          44.    , 253.    ,   5.425 , 163.6   ,   6.    ],
        [ 59.    ,   2.    , 185.    ,  82.5   ,  96.    ,  85.    ,
          54.    , 170.    ,   4.4659,  98.6   ,   3.    ],
        [ 19.    ,   1.    , 173.    ,  75.7   ,  83.    ,  84.    ,
          46.    , 225.    ,   4.7185, 156.6   ,   5.    ],
        [ 58.    ,   2.    , 168.    , 103.6   , 117.    , 109.    ,
          44.    , 166.    ,   4.9488,  93.8   ,   4.    ],
        [ 38.    ,   1.    , 190.    ,  91.7   ,  84.    ,  87.    ,
          42.    , 162.    ,   4.4427, 103.    ,   4.    ],
        [ 57.    ,   2.    , 170.    ,  91.3   , 117.    , 113.    ,
          40.    , 225.    ,   5.9584, 107.6   ,   6.    ],
        [ 57.    ,   2.    , 161.    ,  60.1   , 107.33  , 112.    ,
          41.    , 231.    ,   5.0304, 159.4   ,   5.63  ],
        [ 59.    ,   2.    , 168.    ,  70.8  

**inicializar parametros y modelo**

In [32]:
#parametros
initializer = tf.keras.initializers.LecunNormal()


#modelo
modelo1 = tf.keras.Sequential([
    tf.keras.layers.Dense(1, kernel_initializer=initializer)
])

#perdida
# Pérdida por error cuadrático medio
loss = tf.keras.losses.Huber(delta=1.0)

#algoritmo de optimización
trainer = tf.keras.optimizers.Adam(learning_rate=0.005)


**Entrenamiento**

In [33]:
num_epochs = 500
for epoch in range(num_epochs):
    for X, y in data_iter:
        with tf.GradientTape() as tape:
            l = loss(modelo1(X, training=True), y)
        grads = tape.gradient(l, modelo1.trainable_variables)
        trainer.apply_gradients(zip(grads, modelo1.trainable_variables))
    l = loss(modelo1(train_features), train_labels)
    print(f'época {epoch + 1}, pérdida {l:f}')

época 1, pérdida 172.226013
época 2, pérdida 147.191727
época 3, pérdida 122.176643
época 4, pérdida 97.148827
época 5, pérdida 72.112831
época 6, pérdida 47.095871
época 7, pérdida 22.223131
época 8, pérdida 6.196764
época 9, pérdida 13.135540
época 10, pérdida 9.889647
época 11, pérdida 6.360098
época 12, pérdida 7.853591
época 13, pérdida 6.260232
época 14, pérdida 6.503463
época 15, pérdida 6.103397
época 16, pérdida 6.087951
época 17, pérdida 5.903990
época 18, pérdida 5.891692
época 19, pérdida 5.806950
época 20, pérdida 5.762918
época 21, pérdida 5.715695
época 22, pérdida 5.669023
época 23, pérdida 5.621439
época 24, pérdida 5.575863
época 25, pérdida 5.523025
época 26, pérdida 5.471646
época 27, pérdida 5.426051
época 28, pérdida 5.391691
época 29, pérdida 5.318589
época 30, pérdida 5.268072
época 31, pérdida 5.214972
época 32, pérdida 5.162135
época 33, pérdida 5.134873
época 34, pérdida 5.062383
época 35, pérdida 5.006162
época 36, pérdida 4.951899
época 37, pérdida 4.890641

KeyboardInterrupt: 